<div style='background-color: #ffcccc; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid red;'>
    <h1 style='text-align: center; margin: 0; color: red;'>Problem Stereo Matching</h1>
</div>

<style>
h1 {
  color: #2c3e50;
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  text-align: center;
  padding-bottom: 10px;
  border-bottom: 2px solid #3498db;
}

h2 {
  color: #2980b9;
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
}

p {
  font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
  line-height: 1.6;
  color: #333;
}

ul {
  margin-left: 20px;
  color: #2c3e50;
}

code {
  background-color: #f4f4f4;
  padding: 2px 6px;
  border-radius: 4px;
  font-family: monospace;
}

.box {
  background-color: #ecf0f1;
  padding: 15px;
  border-left: 6px solid #3498db;
  margin: 20px 0;
  border-radius: 8px;
}

</style>

# 📷 Stereo Matching: Trích Xuất Độ Sâu Từ Ảnh

## 🎯 Mục tiêu
<p>Stereo Matching là một kỹ thuật trong thị giác máy tính nhằm ước lượng bản đồ độ sâu (depth map) từ hai ảnh chụp cùng một cảnh từ hai góc nhìn khác nhau (ảnh trái và ảnh phải).</p>

## ⚙️ Các bước chính:

<ul>
  <li><strong>1. Rectification</strong>: Căn chỉnh 2 ảnh sao cho các dòng epipolar nằm ngang.</li>
  <li><strong>2. Matching Cost Computation</strong>: Tính độ tương đồng giữa các pixel hoặc patch.</li>
  <li><strong>3. Cost Aggregation</strong>: Làm mượt và giảm nhiễu chi phí.</li>
  <li><strong>4. Disparity Optimization</strong>: Chọn độ sai khác tốt nhất.</li>
  <li><strong>5. Disparity Refinement</strong>: Cải thiện kết quả bằng lọc và nội suy.</li>
</ul>

## 🧠 Phân loại phương pháp:

<ul>
  <li><strong>Local Methods</strong>: Nhanh, nhưng ít chính xác.</li>
  <li><strong>Global Methods</strong>: Chính xác hơn, nhưng tốn tài nguyên.</li>
  <li><strong>Semi-Global Matching (SGM)</strong>: Cân bằng giữa tốc độ và độ chính xác.</li>
  <li><strong>Deep Learning-based</strong>: Dùng CNN để học tương ứng — ví dụ: <code>GC-Net</code>, <code>PSMNet</code>, <code>RAFT-Stereo</code>.</li>
</ul>

<div class="box">
  📌 <strong>Ứng dụng:</strong>  
  Xe tự lái 🚗, Robot 🤖, AR/VR 🕶, Tái dựng 3D 🧱, Nhận dạng cử chỉ ✋
</div>

## 🧮 Công thức tính độ sâu

<div class="formula-box">
Nếu \( d \) là độ sai khác (disparity), \( f \) là tiêu cự của camera, và \( B \) là khoảng cách giữa hai camera (baseline), thì độ sâu \( Z \) được tính theo công thức:

\\[
Z = \frac{f \cdot B}{d}
\\]

👉 Độ sai khác càng lớn → vật càng gần; disparity nhỏ → vật xa.
</div>

<div style='background-color: #e6f7ff; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #1890ff;'>    
    <h3 style='margin: 0; color: #1890ff;'>ℹ️ Thông tin</h3>
    <p>
       - Trong code bài tập này chúng ta chỉ tập trung vào mục 2 (Matching Cost Computation) và mục 4 (Disparity Decision) trong các theo tác trong stereo matching
<br>
    </p>
</div>

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Pipeline xây dựng giải thuật tính Disparity giữa 2 ảnh trái và phải</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng Method 1: AD cost volume - Using for loop
    </div>
        <div style="margin: 10px 0 0; font-weight: bold;  font-size: 16px;">
    🧮 Công thức:

- **Vector dịch**:
  $$
  \mathbf{d} = \begin{bmatrix} d \\ 0 \end{bmatrix}, \quad d \in D
  $$

- **Chi phí sai lệch tại pixel $p$ với độ lệch d**:
  $$
  C_1(p, \mathbf{d}) = \left| L(p) - R(p - \mathbf{d}) \right|
  $$

- **Tìm độ lệch tối ưu**:
  $$
  d = \underset{d \in D}{\arg\min} \, C_1(p, \mathbf{d})
  $$

$\Rightarrow$  Giá trị $ d $ này chính là **disparity** ứng với pixel $ p $.
    </div>

</div>

In [1]:
import cv2
import numpy as np

def compute_disparity(left_img, right_img, disparity_range):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tạo ra một ma trận chứa disparity giữa hai ảnh (hiệu xL - xR)
    depth = np.zeros((height, width), np.uint8)
    scale = 255 / disparity_range

    for y in range(height):
        for x in range(width):

            disparity = 0;
            cost_min = abs(left[y, x] - right[y, x])
            for d in range(disparity_range):
                if (x - d) < 0:
                    cost = 255;
                else:
                    cost = abs(left[y, x] - right[y, x - d])
                    
                if cost < cost_min:
                    cost_min = cost
                    disparity = d;

            depth[y, x] = disparity * scale

    return depth

if __name__ == "__main__":
    # Đường dẫn ảnh trái và phải
    left_path = "view1_230730.png"
    right_path = "view5_230730.png"
    
    # Đặt khoảng disparity tối đa
    max_disparity = 64

    # Tính bản đồ disparity
    disparity_map = compute_disparity(left_path, right_path, max_disparity)
    
    # Chuyển sang ảnh màu bằng color map (màu giả)
    colored_disparity = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)

    # Hiển thị ảnh màu
    cv2.imshow("Disparity Map (Colored)", colored_disparity)

    # Hiển thị ảnh kết quả
    cv2.imshow("Disparity Map", disparity_map)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu kết quả ra file
    cv2.imwrite("depth_map_1.png", disparity_map)
    cv2.imwrite("depth_map_colored_1.png", colored_disparity)

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Pipeline xây dựng giải thuật tính Disparity giữa 2 ảnh trái và phải</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng Method 1: AD cost volume - Vectorization - Tăng tốc độ tính toán - Code gọn
    </div>
    <div style="margin: 10px 0 0; font-weight: bold;  font-size: 16px;">
    🧮 Công thức:

- **Vector dịch**:
  $$
  \mathbf{d} = \begin{bmatrix} d \\ 0 \end{bmatrix}, \quad d \in D
  $$

- **Chi phí sai lệch tại pixel $p$ với độ lệch d**:
  $$
  C_1(p, \mathbf{d}) = \left| L(p) - R(p - \mathbf{d}) \right|
  $$

- **Tìm độ lệch tối ưu**:
  $$
  d = \underset{d \in D}{\arg\min} \, C_1(p, \mathbf{d})
  $$

$\Rightarrow $ Giá trị $ d $ này chính là **disparity** ứng với pixel $ p $.
    </div>

</div>

In [60]:
import cv2
import numpy as np

def compute_disparity(left_img, right_img, disparity_range):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]
    # compute cost space
    cost_space = np.zeros((height, width, disparity_range))
    for d in range(disparity_range):
        left_d = left[:, d:width]
        right_d = right[:, 0:width - d]
        cost_d = np.abs(left_d - right_d)
        cost_space[:,d:width, d] = cost_d

    # compute disparity from cost space
    scale = 255 / disparity_range
    depth = np.argmin(cost_space, axis = 2)
    depth = depth * scale
    depth = depth.astype(np.uint8)

    return depth

if __name__ == "__main__":
        # Đường dẫn ảnh trái và phải
    left_path = "view1_230730.png"
    right_path = "view5_230730.png"
    
    # Đặt khoảng disparity tối đa
    max_disparity = 64

    # Tính bản đồ disparity
    disparity_map = compute_disparity(left_path, right_path, max_disparity)
    
    # Chuyển sang ảnh màu bằng color map (màu giả)
    colored_disparity = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)

    # Hiển thị ảnh màu
    cv2.imshow("Disparity Map (Colored)", colored_disparity)

    # Hiển thị ảnh kết quả
    cv2.imshow("Disparity Map", disparity_map)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu kết quả ra file
    cv2.imwrite("depth_map_2.png", disparity_map)
    cv2.imwrite("depth_map_colored_2.png", colored_disparity)

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Pipeline xây dựng giải thuật tính Disparity giữa 2 ảnh trái và phải</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng Method 2: SAD cost volume - Using Windown
    </div>
    <div style="margin: 10px 0 0; font-weight: bold;  font-size: 16px;">
🔹 Ký hiệu và định nghĩa

- $ L $: ảnh trái (left image)
- $ R $: ảnh phải (right image)
- $ p = \begin{bmatrix} x_p \\ y_p \end{bmatrix} $: tọa độ pixel đang xét trên ảnh trái
- $ d = \begin{bmatrix} d \\ 0 \end{bmatrix} \in D $: vector disparity dịch theo trục x
- $ W_p $: cửa sổ (window) centered tại $ p $
- $ u \in W_p $: điểm trong cửa sổ quanh $ p $
 
🧮 Công thức:
$$
C_2(p, d) = \sum_{u \in W_p} \left| L(u) - R(u - d) \right|
$$

- Hàm chi phí SAD cộng tất cả sai khác tuyệt đối giữa các điểm tương ứng trong cửa sổ ảnh trái và ảnh phải sau khi dịch.

🔹 Tìm disparity tối ưu tại mỗi pixel:

$$
d = \arg\min_{d \in D} C_2(p, d)
$$

- Với mỗi điểm ảnh $ p $, ta tìm giá trị disparity $ d $ sao cho hàm chi phí nhỏ nhất.
- Gán giá trị $ d $ đó vào disparity map tại vị trí $ p $.

    </div>

</div>

In [98]:
def windown_based_maching_abs(left_img, right_img, disparity_range, kernel_size = 5):
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]

    # Tao ra ma tran disparity_map
    disparity_map = np.zeros((height, width), np.uint8)
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            # Tim j de chi phi la thap nhat
            disparity = 0
            cost_value = 10 ** 5
            for j in range(disparity_range):
                d = x - j
                cost = -1
                if (d - kernel_half) > 0:
                    Sum_AD = 0
                    left_windown = left[(y - kernel_half) : (y + kernel_half) + 1, (x - kernel_half) : (x + kernel_half) + 1]
                    right_windown = right[(y - kernel_half) : (y + kernel_half) + 1, (d - kernel_half) : (d + kernel_half) + 1]

                    left_window_vector = []
                    right_windown_vector = []
                    for i in range(kernel_size):
                        for j in range(kernel_size):
                            left_windown_vector.append(left_windown[i][j])
                            right_windown_vector.append(right_windown[i][j])
                    
                    #left_windown_vector = left_windown.flatten();
                    #right_windown_vector = right_windown.flatten();

                    for k, h in zip(left_windown_vector, right_windown_vector):
                        Sum_AD += abs(k - h)

                    cost = Sum_AD
                    if cost < cost_value:
                        cost_value = cost
                        disparity = j

            disparity_map[y,x] = disparity * scale
                
    return disparity_map
if __name__ == "__main__":
        # Đường dẫn ảnh trái và phải
    left_path = "view1_230730.png"
    right_path = "view5_230730.png"
    
    # Đặt khoảng disparity tối đa
    max_disparity = 64

    # Tính bản đồ disparity
    disparity_map = windown_based_maching_abs(left_path, right_path, max_disparity, kernel_size = 5)
    
    # Chuyển sang ảnh màu bằng color map (màu giả)
    colored_disparity = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)

    # Hiển thị ảnh màu
    cv2.imshow("Disparity Map (Colored)", colored_disparity)

    # Hiển thị ảnh kết quả
    cv2.imshow("Disparity Map", disparity_map)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu kết quả ra file
    cv2.imwrite("depth_map_3.png", disparity_map)
    cv2.imwrite("depth_map_colored_3.png", colored_disparity)

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Pipeline xây dựng giải thuật tính Disparity giữa 2 ảnh trái và phải</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng Method 3: CCS cost volume - Using Windown
    </div>
    <div style="margin: 10px 0 0; font-weight: bold;  font-size: 16px;">
🔹 Ký hiệu và định nghĩa

- $ L $: ảnh trái (left image)
- $ R $: ảnh phải (right image)
- $ p = \begin{bmatrix} x_p \\ y_p \end{bmatrix} $: tọa độ pixel đang xét trên ảnh trái
- $ d = \begin{bmatrix} d \\ 0 \end{bmatrix} \in D $: vector disparity dịch theo trục x
- $ W_p $: cửa sổ (window) centered tại $ p $
  
🧮 Công thức:
$$
C_3(p, d) = CS (L(W_p), R(W_(p-d))
$$

- Hàm chi phí CCS tinh do tuong dong giữa cửa sổ ảnh trái và ảnh phải sau khi dịch.

🔹 Tìm disparity tối ưu tại mỗi pixel:

$$
d = \arg\max_{d \in D} C_3(p, d)
$$

- Với mỗi điểm ảnh $ p $, ta tìm giá trị disparity $ d $ sao cho hàm chi phí lon nhất.
- Gán giá trị $ d $ đó vào disparity map tại vị trí $ p $.

    </div>

</div>

In [3]:
import numpy as np
import cv2

from math import *
def compute_cosine(vector1, vector2):
    tich_vo_huong = 0;
    for x, y in zip(vector1, vector2):
        tich_vo_huong += x * y;
    norm_vector1 = 0;
    norm_vector2 = 0;
    for x in vector1:
        norm_vector1 += x ** 2
    for y in vector2:
        norm_vector2 += y ** 2
    norm_vector1 = (norm_vector1) ** 0.5
    norm_vector2 = (norm_vector2) ** 0.5
    tich_do_dai = norm_vector1 * norm_vector2
    cosine_value = tich_vo_huong / tich_do_dai
    return cosine_value

def window_based_matching_cosine(left_img, right_img, disparity_range, kernel_size = 5):
    # doc va xu ly anh
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]
    
    # Tao ma tran Disparity
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3;
    disparity_map = np.zeros((height, width), np.uint8)

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_max = -99999
            for j in range(disparity_range):
                x_r = x - j
                cost = 0
                if (x_r - kernel_half) > 0:
                    left_windown = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    right_windown = right[(y - kernel_half):(y + kernel_half) + 1, (x_r - kernel_half) : (x_r + kernel_half) + 1]

                    '''
                    left_windown_flatten = []
                    right_windown_flatten = []
                    for i in range(k ernel_size):
                        for j in range(kernel_size):
                            left_windown_flatten.append(left_windown[i][j])
                            right_windown_flatten.append(right_windown[i][j])
                    '''
                    left_windown_flatten = left_windown.flatten();
                    right_windown_flatten = right_windown.flatten();

                    cost = compute_cosine(left_windown_flatten, right_windown_flatten)

                    if cost > cost_max:
                        cost_max = cost
                        disparity = j

            disparity_map[y, x] = disparity * scale
            
    return disparity_map
if __name__ == "__main__":
        # Đường dẫn ảnh trái và phải
    left_path = "view1_230730.png"
    right_path = "view5_230730.png"
    
    # Đặt khoảng disparity tối đa
    max_disparity = 64

    # Tính bản đồ disparity
    disparity_map = window_based_matching_cosine(left_path, right_path, max_disparity, kernel_size = 5)
    
    # Chuyển sang ảnh màu bằng color map (màu giả)
    colored_disparity = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)

    # Hiển thị ảnh màu
    cv2.imshow("Disparity Map (Colored)", colored_disparity)

    # Hiển thị ảnh kết quả
    cv2.imshow("Disparity Map", disparity_map)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu kết quả ra file
    cv2.imwrite("depth_map_4.png", disparity_map)
    cv2.imwrite("depth_map_colored_4.png", colored_disparity)

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Pipeline xây dựng giải thuật tính Disparity giữa 2 ảnh trái và phải</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng Method 4: Correlation Cofficient cost volume - Using Windown
    </div>
    <div style="margin: 10px 0 0; font-weight: bold;  font-size: 16px;">
🔹 Ký hiệu và định nghĩa

- $ L $: ảnh trái (left image)
- $ R $: ảnh phải (right image)
- $ p = \begin{bmatrix} x_p \\ y_p \end{bmatrix} $: tọa độ pixel đang xét trên ảnh trái
- $ d = \begin{bmatrix} d \\ 0 \end{bmatrix} \in D $: vector disparity dịch theo trục x
- $ W_p $: cửa sổ (window) centered tại $ p $
  
🧮 Công thức:
$$
C_4(p, d) = CR (L(W_p), R(W_(p-d))
$$

Hệ số tương quan CR được tính bằng công thức:
$$
r = \frac{n \sum x_i y_i - \sum x_i \sum y_i}
         {\sqrt{(n \sum x_i^2 - (\sum x_i)^2)(n \sum y_i^2 - (\sum y_i)^2)}}
$$
- Hàm chi phí CR tinh do tuong dong giữa cửa sổ ảnh trái và ảnh phải sau khi dịch.

🔹 Tìm disparity tối ưu tại mỗi pixel:

$$
d = \arg\max_{d \in D} C_4(p, d)
$$

- Với mỗi điểm ảnh $ p $, ta tìm giá trị disparity $ d $ sao cho hàm chi phí lon nhất.
- Gán giá trị $ d $ đó vào disparity map tại vị trí $ p $.

    </div>

</div>

In [122]:
def compute_Correlation_Cofficient(vector1, vector2):
    n = len(vector1)
    tong_cua_tich = 0
    sum_x = 0
    sum_x_2 = 0
    sum_y = 0
    sum_y_2 = 0
    for x, y in zip(vector1, vector2):
        tong_cua_tich += x * y
    for x in vector1:
        sum_x += x
        sum_x_2 += x ** 2
    for y in vector2:
        sum_y += y
        sum_y_2 += y ** 2
    cov = n * tong_cua_tich - sum_x * sum_y
    sigma_1 = n * sum_x_2 - sum_x ** 2
    sigma_2 = n * sum_y_2 - sum_y ** 2
    he_so_tuong_quan = cov / sqrt(sigma_1 * sigma_2)

    return he_so_tuong_quan

def window_based_matching_Correlation_Cofficient(left_img, right_img, disparity_range, kernel_size = 5):
    # doc va xu ly anh
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]
    
    # Tao ma tran Disparity
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3;
    disparity_map = np.zeros((height, width), np.uint8)

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_max = -99999
            for j in range(disparity_range):
                x_r = x - j
                cost = 0
                if (x_r - kernel_half) > 0:
                    left_windown = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    right_windown = right[(y - kernel_half):(y + kernel_half) + 1, (x_r - kernel_half) : (x_r + kernel_half) + 1]
                    '''
                    left_windown_flatten = []
                    right_windown_flatten = []
                    for i in range(kernel_size):
                        for j in range(kernel_size):
                            left_windown_flatten.append(left_windown[i][j])
                            right_windown_flatten.append(right_windown[i][j])
                    '''
                    
                    left_windown_flatten = left_windown.flatten()
                    right_windown_flatten = right_windown.flatten()
                    
                    cost = compute_Correlation_Cofficient(left_windown_flatten, right_windown_flatten)

                    if cost > cost_max:
                        cost_max = cost
                        disparity = j

            disparity_map[y, x] = disparity * scale
            
    return disparity_map
if __name__ == "__main__":
        # Đường dẫn ảnh trái và phải
    left_path = "view1_230730.png"
    right_path = "view5_230730.png"
    
    # Đặt khoảng disparity tối đa
    max_disparity = 64

    # Tính bản đồ disparity
    disparity_map = window_based_matching_Correlation_Cofficient(left_path, right_path, max_disparity, kernel_size = 5)
    
    # Chuyển sang ảnh màu bằng color map (màu giả)
    colored_disparity = cv2.applyColorMap(disparity_map, cv2.COLORMAP_JET)

    # Hiển thị ảnh màu
    cv2.imshow("Disparity Map (Colored)", colored_disparity)

    # Hiển thị ảnh kết quả
    cv2.imshow("Disparity Map", disparity_map)
    
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    # Lưu kết quả ra file
    cv2.imwrite("depth_map_5.png", disparity_map)
    cv2.imwrite("depth_map_colored_5.png", colored_disparity)
    

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Question 5 - Stereo Matching based on Cosine Cost</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng phương pháp đo lường Cosine cho Matching Cost
    </div>
</div>

In [19]:
from math import *
def compute_cosine_q5(vector1, vector2):
    '''
    tich_vo_huong = 0
    for x, y in zip(vector1, vector2):
        tich_vo_huong += x * y;
    norm_vector1 = 0
    norm_vector2 = 0
    for x in vector1:
        norm_vector1 += x ** 2
    for y in vector2:
        norm_vector2 += y ** 2
    norm_vector1 = norm_vector1 ** 0.5 
    norm_vector2 = norm_vector2 ** 0.5
    tich_do_dai = norm_vector1 * norm_vector2
    cosine_value = tich_vo_huong / tich_do_dai
    '''
    v1 = np.array(vector1)
    v2 = np.array(vector2)
    cosine_value = np.dot(vector1, vector2) / (np.linalg.norm(vector1) * np.linalg.norm(vector2) + 1e-8)

    return cosine_value

def window_based_matching_cosine_q5(left_img, right_img, disparity_range, kernel_size = 5):
    # doc va xu ly anh
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]
    
    # Tao ma tran Disparity
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3;
    disparity_map = np.zeros((height, width), np.uint8)

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_max = -99999
            for j in range(disparity_range):
                x_r = x - j
                cost = 0
                if (x_r - kernel_half) > 0:
                    left_windown = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    right_windown = right[(y - kernel_half):(y + kernel_half) + 1, (x_r - kernel_half) : (x_r + kernel_half) + 1]

                    '''
                    left_windown_flatten = []
                    right_windown_flatten = []
                    for i in range(kernel_size):
                        for j in range(kernel_size):
                            left_windown_flatten.append(left_windown[i][j])
                            right_windown_flatten.append(right_windown[i][j])
                    '''
                    left_windown_flatten = left_windown.flatten()
                    right_windown_flatten = right_windown.flatten()
                    
                    cost = compute_cosine_q5(left_windown_flatten, right_windown_flatten)

                    if cost > cost_max:
                        cost_max = cost
                        disparity = j

            disparity_map[y, x] = disparity * scale
            
    return disparity_map

In [21]:
#Question 5

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "view1_230730.png"
right_img_path = "view5_230730.png"
groundtruth_img_path = "disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine_q5(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

rmse : 4.987600619253398


<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Question 6 - Stereo Matching based on Correlation Cost</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Sử dụng phương pháp đo lường Correlation cho Matching Cost
    </div>
</div>

In [7]:
import numpy as np
import cv2

def compute_Correlation_Cofficient_q6(vector1, vector2):
    n = len(vector1)
    tong_cua_tich = 0
    sum_x = 0
    sum_x_2 = 0
    sum_y = 0
    sum_y_2 = 0
    for x, y in zip(vector1, vector2):
        tong_cua_tich += x * y
    for x in vector1:
        sum_x += x
        sum_x_2 += x ** 2
    for y in vector2:
        sum_y += y
        sum_y_2 += y ** 2
    cov = n * tong_cua_tich - sum_x * sum_y
    sigma_1 = n * sum_x_2 - sum_x ** 2
    sigma_2 = n * sum_y_2 - sum_y ** 2
    he_so_tuong_quan = cov / ((sigma_1 * sigma_2 + 1e-5) ** 0.5)

    return he_so_tuong_quan

def window_based_matching_Correlation_Cofficient_q6(left_img, right_img, disparity_range, kernel_size = 5):
    # doc va xu ly anh
    left = cv2.imread(left_img, 0)
    right = cv2.imread(right_img, 0)

    left = left.astype(np.float32)
    right = right.astype(np.float32)

    height, width = left.shape[:2]
    
    # Tao ma tran Disparity
    kernel_half = int((kernel_size - 1) / 2)
    scale = 3;
    disparity_map = np.zeros((height, width), np.uint8)

    for y in range(kernel_half, height - kernel_half):
        for x in range(kernel_half, width - kernel_half):
            disparity = 0
            cost_max = -99999
            for j in range(disparity_range):
                x_r = x - j
                cost = 0
                if (x_r - kernel_half) > 0:
                    left_windown = left[(y - kernel_half):(y + kernel_half) + 1, (x - kernel_half):(x + kernel_half) + 1]
                    right_windown = right[(y - kernel_half):(y + kernel_half) + 1, (x_r - kernel_half) : (x_r + kernel_half) + 1]
                    '''
                    left_windown_flatten = []
                    right_windown_flatten = []
                    for i in range(kernel_size):
                        for j in range(kernel_size):
                            left_windown_flatten.append(left_windown[i][j])
                            right_windown_flatten.append(right_windown[i][j])
                    '''
                    
                    left_windown_flatten = left_windown.flatten()
                    right_windown_flatten = right_windown.flatten()
                    
                    cost = compute_Correlation_Cofficient_q6(left_windown_flatten, right_windown_flatten)

                    if cost > cost_max:
                        cost_max = cost
                        disparity = j

            disparity_map[y, x] = disparity * scale
            
    return disparity_map

In [9]:
#Question 6
def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "view1_230730.png"
right_img_path = "view5_230730.png"
groundtruth_img_path = "disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_Correlation_Cofficient_q6(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

rmse : 5.289219155748246


<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Question 7 - Stereo Matching based on Cosine Cost</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Tinh toan do loi trong truong hop 2 anh trai va phai khac nhau ve do sang
    </div>
</div>

In [ ]:
#Question 7

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "view1_d.png"
right_img_path = "view5_d.png"
groundtruth_img_path = "disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_cosine(
    left_img_path,
    right_img_path, 
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)

<div style="background-color: #fffbe6; padding: 15px; border-radius: 5px; margin-bottom: 20px; color: black; border-left: 5px solid #faad14;">    
    <h3 style="margin: 0; color: #faad14;">📌 Question 8 - Stereo Matching based on Correlation Cofficient Cost</h3>
    <div style="margin: 10px 0 0; font-weight: bold; text-align: center; font-size: 16px;">
        Tinh toan do loi trong truong hop 2 anh trai va phai khac nhau ve do sang
    </div>
</div>

In [ ]:
#Question 8

def compute_rms_error(matrix1, matrix2):
  return np.sqrt(np.sum(np.square(matrix1-matrix2))/(matrix1.shape[0]*matrix1.shape[1]))

left_img_path = "view1_d.png"
right_img_path = "view5_d.png"
groundtruth_img_path = "disp1.png"

disparity_range = 64
kernel_size = 5

depth = window_based_matching_correlation(
    left_img_path,
    right_img_path,
    disparity_range,
    kernel_size=kernel_size)

groundtruth_image = cv2.imread(groundtruth_img_path, 0)
rmse = compute_rms_error(depth, groundtruth_image)
print("rmse :", rmse)